In [ ]:
from pymongo import MongoClient
import urllib.parse
import pprint
username = urllib.parse.quote_plus('jayantj1j')

client=MongoClient("mongodb://%s:%s@mflix-shard-00-00-sgqip.mongodb.net:27017,mflix-shard-00-01-sgqip.mongodb.net:27017,mflix-shard-00-02-sgqip.mongodb.net:27017/mflix?ssl=true&replicaSet=mflix-shard-0&authSource=admin&retryWrites=true" % (username, password))
print(client.mflix.comments)

In [ ]:
pipeline = [
    {
        '$limit': 100
    },
    {
        '$project': {
            'title': 1,
            'year': 1,
            'directors': {'$split': ["$director", ", "]},
            'actors': {'$split': ["$cast", ", "]},
            'writers': {'$split': ["$writer", ", "]},
            'genres': {'$split': ["$genre", ", "]},
            'languages': {'$split': ["$language", ", "]},
            'countries': {'$split': ["$country", ", "]},
            'plot': 1,
            'fullPlot': "$fullplot",
            'rated': "$rating",
            'released': 1,
            'runtime': 1,
            'poster': 1,
            'imdb': {
                'id': "$imdbID",
                'rating': "$imdbRating",
                'votes': "$imdbVotes"
                },
            'metacritic': 1,
            'awards': 1,
            'type': 1,
            'lastUpdated': "$lastupdated"
        }
    },
    {
        '$out': "movies_scratch"
    }
]

pprint.pprint(list(client.mflix.movies_initial.aggregate(pipeline)))

In [ ]:
pipeline2 = [
    {
        '$limit': 100
    },
    
    
    {
        "$facet":{ 'original':[{ '$limit': 100}],
                 
                
                    'shortening':[{'$project': {
            'title': 1,
            'year': 1,
            'directors': {'$split': ["$director", ", "]},
            'actors': {'$split': ["$cast", ", "]},
            'writers': {'$split': ["$writer", ","]},
            'genres': {'$split': ["$genre", ", "]},
            'languages': {'$split': ["$language", ", "]},
            'countries': {'$split': ["$country", ", "]},
            'plot': 1,
            'fullPlot': "$fullplot",
            'rated': "$rating",
            'released': {
                '$cond':{
                    'if':{'$ne':["$released",""]},
                    'then':{
                        'dateFromString':{
                            'dateString':"$released"
                        }
                    },'else':""}},
          
            'runtime': 1,
            'poster': 1,
            'imdb': {
                'id': "$imdbID",
                'rating': "$imdbRating",
                'votes': "$imdbVotes"
                },
            'metacritic': 1,
            'awards': 1,
            'type': 1,
            'lastUpdated': "$lastupdated"
        
                    
        
    }}]
                }

    }]

pprint.pprint(list(client.mflix.movies_initial.aggregate(pipeline2)))

In [ ]:
pipelineff = [
    {
        '$limit': 100
    },{
        '$addFields':{
            'lastupdated':{
                '$arrayElemAt':[
                    
                    
                    {'$split':["$lastupdated","."]},0]
                
            }
        }
        
        
    },
    {
        '$project': {
            'title': 1,
            'year': 1,
            'directors': {'$split': ["$director", ", "]},
            'actors': {'$split': ["$cast", ", "]},
            'writers': {'$split': ["$writer", ", "]},
            'genres': {'$split': ["$genre", ", "]},
            'languages': {'$split': ["$language", ", "]},
            'countries': {'$split': ["$country", ", "]},
            'plot': 1,
            'fullPlot': "$fullplot",
            'rated': "$rating",
             'released': {
                '$cond':{
                    'if':{'$ne':["$released",""]},
                    'then':{
                        'dateFromString':{
                            'dateString':"$released"
                        }
                    },'else':""}},
          
            'runtime': 1,
            'poster': 1,
            'imdb': {
                'id': "$imdbID",
                'rating': "$imdbRating",
                'votes': "$imdbVotes"
                },
            'metacritic': 1,
            'awards': 1,
            'type': 1,
            'lastUpdated': {
                '$cond':{
                    'if':{'$ne':["$lastupdated",""]},
                    "then":{
                        "$dateFromString":{
                            
                            'dateString':'$lastupdated',
                            'timezone':"America/New_York"
                            
                        }
                        
                        
                    },"else":""
                    
                }
                
            }
        }
    },
    {
        '$out': "movies_scratch_project1"
    }
]

pprint.pprint(list(client.mflix.movies_initial.aggregate(pipelineff)))

In [ ]:
from datetime import datetime
import re

In [ ]:
runtime_pat=re.compile(r'([0-9]+) min')

In [ ]:

for movie in client.mflix.movies_initial.find({}).limit(30):

    fields_to_set = {}
    fields_to_unset = {}
    for k,v in movie.copy().items():
        if v=="" or v==[""]:
            del movie[k]
            fields_to_unset[k]=""
            
    if 'director' in movie:
        fields_to_unset['director']=""
        fields_to_set["directors"]=movie["director"].split(",")
    if 'cast' in movie:
        fields_to_set['cast'] = movie['cast'].split(", ")
        
    if 'writer' in movie:
        fields_to_unset['writer'] = ""
        fields_to_set['writers'] = movie['writer'].split(", ")
    if 'genre' in movie:
        fields_to_unset['genre'] = ""
        fields_to_set['genres'] = movie['genre'].split(", ")
    if 'language' in movie:
        fields_to_unset['language'] = ""
        fields_to_set['languages'] = movie['language'].split(", ")
    if 'country' in movie:
        fields_to_unset['country'] = ""
        fields_to_set['countries'] = movie['country'].split(", ")
        
    if 'fullplot' in movie:
        fields_to_unset['fullplot']=""
        fields_to_set["fullplot"]=movie["fullplot"]
    if 'rating' in movie:
        fields_to_unset['rating']=""
        fields_to_set["rated"]=movie['rating']
        
    imdb={}    
    if 'imdbID' in movie:
        fields_to_unset['imdbID']=""
        imdb["id"]=movie["imdbID"]
        
    if 'imdbRating' in movie:
        fields_to_unset['imdbRating'] = ""
        imdb['rating'] = movie['imdbRating']
    if 'imdbVotes' in movie:
        fields_to_unset['imdbVotes'] = ""
        imdb['votes'] = movie['imdbVotes']
    if imdb:
        fields_to_set['imdb'] = imdb
    if "released" in movie:
        fields_to_set['released'] = datetime.strptime(movie['released'],
                                                  "%Y-%m-%d")
    if 'lastUpdated' in movie:
        fields_to_set['lastUpdated'] = datetime.strptime(movie['lastUpdated'][0:19],
                                                         "%Y-%m-%d %H:%M:%S")
    
    
    if 'runtime' in movie:
        m=runtime_pat.match(movie['runtime'])
        if m:
            fields_to_set['runtime'] = int(m.group(1))
    
    update_doc={}
    if fields_to_set:
        update_doc["$set"]=fields_to_set
    if fields_to_unset:
        update_doc["$unset"]=fields_to_unset
    pprint.pprint(update_doc)
    
   # client.mflix.update_one({'_id': movie['_id']}, update_doc)


In [ ]:
from pymongo import MongoClient, UpdateOne
from datetime import datetime



runtime_pat = re.compile(r'([0-9]+) min')

batch_size = 10
updates = []
count = 0
for movie in client.mflix.movies_initial.find({}):
    
    fields_to_set = {}
    fields_to_unset = {}

    for k,v in movie.copy().items():
        if v == "" or v == [""]:
            del movie[k]
            fields_to_unset[k] = ""

    if 'director' in movie:
        fields_to_unset['director'] = ""
        fields_to_set['directors'] = movie['director'].split(", ")
    if 'cast' in movie:
        fields_to_set['cast'] = movie['cast'].split(", ")
    if 'writer' in movie:
        fields_to_unset['writer'] = ""
        fields_to_set['writers'] = movie['writer'].split(", ")
    if 'genre' in movie:
        fields_to_unset['genre'] = ""
        fields_to_set['genres'] = movie['genre'].split(", ")
    if 'language' in movie:
        fields_to_unset['language'] = ""
        fields_to_set['languages'] = movie['language'].split(", ")
    if 'country' in movie:
        fields_to_unset['country'] = ""
        fields_to_set['countries'] = movie['country'].split(", ")
        
    if 'fullplot' in movie:
        fields_to_unset['fullplot'] = ""
        fields_to_set['fullPlot'] = movie['fullplot']
    if 'rating' in movie:
        fields_to_unset['rating'] = ""
        fields_to_set['rated'] = movie['rating']

    imdb = {}
    if 'imdbID' in movie:
        fields_to_unset['imdbID'] = ""
        imdb['id'] = movie['imdbID']
    if 'imdbRating' in movie:
        fields_to_unset['imdbRating'] = ""
        imdb['rating'] = movie['imdbRating']
    if 'imdbVotes' in movie:
        fields_to_unset['imdbVotes'] = ""
        imdb['votes'] = movie['imdbVotes']
    if imdb:
        fields_to_set['imdb'] = imdb
        
#     if 'released' in movie and "released"!=' Short"':
#         fields_to_set['released'] = datetime.strptime(movie['released'], "%Y-%m-%d")
                                                     
    if 'lastUpdated' in movie:
        fields_to_set['lastUpdated'] = datetime.strptime(movie['lastUpdated'][0:19], "%Y-%m-%d %H:%M:%S")
                                                        

    if 'runtime' in movie:
        m = runtime_pat.match(movie['runtime']) 
        if m:
            fields_to_set['runtime'] = int(m.group(1))

    update_doc = {}
    if fields_to_set:
        update_doc['$set'] = fields_to_set
    if fields_to_unset:
        update_doc['$unset'] = fields_to_unset

    updates.append(UpdateOne({'_id': movie['_id']}, update_doc))

    count += 1
    if count == batch_size:
        client.mflix.movies.bulk_write(updates)
        updates = []
        break




In [ ]:
import urllib.parse


db=MongoClient(("mongodb+srv://jayantj1j:%s@mflix-sgqip.mongodb.net/test?retryWrites=true") % (password))["mflix"]


In [ ]:
from pymongo import MongoClient, DESCENDING
sort_key = "tomatoes.viewer.numReviews"
filters = {}
page = 0
movies_per_page = 20
# first collect all movies based on passed filters
if "$text" in filters:
    score_meta_doc = { "$meta": "textScore" }
    movies = db.movies.find(filters, { "score": score_meta_doc }).sort([("score", score_meta_doc)])
                      
else:
    movies = db.movies.find(filters).sort(sort_key, DESCENDING)
                      

# count number of total movie documents
total_num_movies = movies.count()

# limit records based on page number
movies = movies.skip(movies_per_page * page).limit(movies_per_page)
             

  


In [ ]:
movie_list = list(movies)
movie_list[0]

In [ ]:
len(list(movies))

In [ ]:
filters={'year':{'$gte':1983}}
for movie in client.mflix.movies_initial.find(filters):
    pprint.pprint(movie['title'])

In [ ]:
# find all movies between 1983 and 1999
filters={'year':{'$gte':1983,'$lt':2000}}
for movie in client.mflix.movies_initial.find(filters):
    pprint.pprint(movie['title'])

In [ ]:
filters1 = { 'year': { '$in': [ 1995, 2005, 2015 ] } }


In [ ]:
filter2={'year':{'$in':[2005,2015]},'genre':{'$ne':'Adult'}}

In [ ]:
for movie in client.mflix.movies_initial.find(filter2):
    pprint.pprint(movie['title'])

In [ ]:
filter3={"comments":{"$exists":True}}
projection={"comments":1}
movie=client.mflix.movies_initial.find_one(filter3,projection)
pprint.pprint(movie)

In [ ]:
import dateparser
betterfilter={
    "comments":{
        "$elemMatch":{
            "name":"Samuel Tarley",
            "date":{
                "$lt":dateparser.parse("1995-01-01")
            }
        }
    }
}

In [ ]:
client=client.mflix.comments

In [ ]:
movie=client.mflix.movies_initial.find_one(betterfilter,projection)
pprint.pprint(movie)